# Fetch data from wekeo
---

First we need to complete the following request too get the authentication token.

> __$ curl --request GET --header 'authorization: Basic <credentials>' https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken__


Next we have to accept the terms and conditions using the authentication token.

> __$ curl --request PUT --header 'accept: application/json' --header 'authorization: <access_token>' --data 'accepted=true' https://'wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/termsaccepted/Copernicus_General_License__

Ultimately we can get the subsetted dataset via the following command.

> __$ curl --request GET --header 'authorization: <access_token>' https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/<datasetId>__

In [23]:
import requests
import base64
import json
import time
from config import credentials

In [4]:
url_token = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken"
res_token = requests.get(url_token, auth=credentials)

access_token = res_token.json()["access_token"]


print("The request token is: ", access_token)

The request token is:  6c5fee3a-0cfd-3627-b56f-577ce7dadfd2


In [5]:
# This is the alternative method to get the token.
api_string = credentials[0] +':'+credentials[1] 
api_key = (base64.b64encode(str.encode(api_string), altchars=None)).decode()

headers = {'Authorization': 'Basic ' + api_key}
response = requests.get(url_token, headers=headers)
access_token = json.loads(response.text)['access_token']

print("The request token is: ", access_token)

The request token is:  6c5fee3a-0cfd-3627-b56f-577ce7dadfd2


After succesfully getting the access token we have to accept the terms and conditions of the wekeo page. First we check if we already have accepted them. If they have been accepted we move on with the next step. If not we accept them with a put request. 

In [17]:
headers = {'Authorization': 'Bearer ' + access_token, 'Accept': 'application/json'}

url_tc = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/termsaccepted/Copernicus_General_License"
res_tc = requests.get(url_tc, headers=headers)

if res_tc.status_code != requests.codes.ok:
    print("Problem! The request returned an error.")
else:
    if json.loads(res_tc.text)["accepted"] is False:
        print("The Terms and Conditions have not been accepted yet.")

        accept_tc = requests.put(url_tc, headers=headers)

        print(f"The Terms and Conditions have been accepted ({json.loads(accept_tc.text)['accepted']}).")

    else:
        print("The terms and conditions were already accepted.")

The terms and conditions were already accepted.


Below we will be using a sample query that is in the documentation of the [Wekeo](https://www.wekeo.eu/docs/harmonised-data-access-api) page. The following query will post a request and continiously check on the query status. When the query is completed the data can be downloaded following the next code chunk.

In [18]:
data = {
  "datasetId": "EO:ESA:DAT:SENTINEL-1:SAR",
  "boundingBoxValues": [
    {
      "name": "bbox", "bbox": [1.13, 43.9, 1.53, 43.68]
      }
    ],
  "dateRangeSelectValues": [
    {
      "name": "dtrange",
      "start": "2020-01-01T00:00:00.000Z",
      "end": "2020-01-12T00:00:00.000Z"
      }
    ],
  "stringChoiceValues": [
    { 
      "name": "producttype", "value": "GRD" 
      }
    ]
  }


In [26]:
t_step = 5 # timer steps in seconds.
t_max = 60 # max timer length in seconds.

url_dataquery = "https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest"
res_post = requests.post(url_dataquery, headers={'content-type': 'application/json', 'authorization': res_token.json()["access_token"]}, json=data)

if res_post.status_code is requests.codes.ok:
        query_id = json.loads(res_post.text)['jobId']
        print ("Query successfully submitted. The query ID is " + query_id)

else:
        job_id = ""
        print(f"Error: Unexpected response {response}")


status = "not started"
message_id = 0
while (status != "completed"):
        message_id = message_id+1

        # Set the response timer.
        wait_timer = min(t_step, t_max)
        print(f"The next status check is in {wait_timer}") 
        time.sleep(wait_timer)

        check_response = requests.get(url_dataquery + "/status/" + query_id, headers=headers)


        if check_response.status_code == requests.codes.ok:
            status = json.loads(check_response.text)["status"]

            print ("The query was succesfully submitted. The status is " + status)
        else:
            print(f"Error: Unexpected response {check_response}")

Query successfully submitted. The query ID is vroJgJf1avbBJSszD4z3Zis4x8U
The next status check is in 5


In the following bit the pervious query is download as zip file.  

In [ ]:
r = requests.get(url, headers=headers, stream=True)